In [74]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

In [2]:
input_file = pd.read_csv(r"E:\Interview\20211030 black_coffer\Input.csv")
input_file = input_file.dropna()
input_file['URL_ID'] =input_file['URL_ID'].astype(int)
input_file['URL_ID'] =input_file['URL_ID'].astype(str)
input_file

,URL_ID,URL
0,37,https://insights.blackcoffer.com/ai-in-healthc...
1,38,https://insights.blackcoffer.com/what-if-the-c...
2,39,https://insights.blackcoffer.com/what-jobs-wil...
3,40,https://insights.blackcoffer.com/will-machine-...
4,41,https://insights.blackcoffer.com/will-ai-repla...
...,...,...
109,146,https://insights.blackcoffer.com/blockchain-fo...
110,147,https://insights.blackcoffer.com/the-future-of...
111,148,https://insights.blackcoffer.com/big-data-anal...
112,149,https://insights.blackcoffer.com/business-anal...


# Data Extraction

In [3]:
url = input_file["URL"][0]
def data_extraction(filename, url):
    headers = {
        "User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:60.0) Gecko/20100101 Firefox/60.0"
    }
    reqs = requests.get(url, headers= headers)
    soup  = BeautifulSoup(reqs.text,'html.parser')
    title = soup.title.string
    p_tags = soup.find_all("p")
    text = ""
    for i in p_tags:
        text += i.get_text()
    article = title +"\n"+ text
    filename = "text_files/"+filename + ".txt"
    with open(filename,"w", encoding = "utf-8") as file:
        file.write(article)

In [4]:
for _, row in input_file.iterrows():
    name = row["URL_ID"]
    url = row["URL"]
    data_extraction(name,url)

# Data Analysis

In [3]:
output_file = pd.DataFrame()
output_file = output_file.append(input_file)

In [4]:
output_file

,URL_ID,URL
0,37,https://insights.blackcoffer.com/ai-in-healthc...
1,38,https://insights.blackcoffer.com/what-if-the-c...
2,39,https://insights.blackcoffer.com/what-jobs-wil...
3,40,https://insights.blackcoffer.com/will-machine-...
4,41,https://insights.blackcoffer.com/will-ai-repla...
...,...,...
109,146,https://insights.blackcoffer.com/blockchain-fo...
110,147,https://insights.blackcoffer.com/the-future-of...
111,148,https://insights.blackcoffer.com/big-data-anal...
112,149,https://insights.blackcoffer.com/business-anal...


In [5]:
def trash_remover(list):
    """
    returns list elements without additional information
    after the typographical symbol pipe (|)
    and transform list in lower case
    """
    for i in range(len(list)):
        list[i] = list[i].lower()
        for j in range(len(list[i])):
            if list[i][j] == "|":
                list[i] = list[i][:j].strip()
                break
    return list

## Stopwords

In [6]:
# making an empty list to store all stopwords
stopwords = []

In [7]:
with open('StopWords\StopWords_Auditor.txt','r',encoding = 'utf-8') as file:
    auditor_sw = file.read()
    auditor_sw = auditor_sw.split('\n')
    auditor_sw = trash_remover(auditor_sw)
for i in auditor_sw:
    stopwords.append(i)

In [8]:
with open('StopWords\StopWords_Currencies.txt','r',encoding = 'utf-8') as file:
    currencies_sw = file.read()
    currencies_sw = currencies_sw.split('\n')
    currencies_sw = trash_remover(currencies_sw)
for i in currencies_sw:
    stopwords.append(i)

In [9]:
with open('StopWords\StopWords_DatesandNumbers.txt','r',encoding = 'utf-8') as file:
    datesnum_sw = file.read()
    datesnum_sw = datesnum_sw.split('\n')
    datesnum_sw = trash_remover(datesnum_sw)
for i in datesnum_sw:
    stopwords.append(i)

In [10]:
with open('StopWords\StopWords_Generic.txt','r',encoding = 'utf-8') as file:
    generic_sw = file.read()
    generic_sw = generic_sw.split('\n')
    generic_sw = trash_remover(generic_sw)
for i in generic_sw:
    stopwords.append(i)

In [11]:
with open('StopWords\StopWords_GenericLong.txt','r',encoding = 'utf-8') as file:
    genericlong_sw = file.read()
    genericlong_sw = genericlong_sw.split('\n')
    genericlong_sw = trash_remover(genericlong_sw)
for i in genericlong_sw:
    stopwords.append(i)

In [12]:
with open('StopWords\StopWords_Geographic.txt','r',encoding = 'utf-8') as file:
    geographic_sw = file.read()
    geographic_sw = geographic_sw.split('\n')
    geographic_sw = trash_remover(geographic_sw)
for i in geographic_sw:
    stopwords.append(i)

In [13]:
with open('StopWords\StopWords_Names.txt','r',encoding = 'utf-8') as file:
    names_sw = file.read()
    names_sw = names_sw.split('\n')
    names_sw = trash_remover(names_sw)
for i in names_sw:
    stopwords.append(i)

In [14]:
len(stopwords)

14109

In [15]:
# stopwords

### Removing Stop Words

In [16]:
def remove_stopwords(text_file,stop_words,output_file):
    with open(text_file,'r') as file:
        text = file.read()
    words = text.split()

    # creating empty list for filtered words
    filter_words = []

    for word in words:
        if word not in stop_words:
            filter_words.append(word)

    filtered_text = " ".join(filter_words)

    with open(output_file, "w") as file:
        file.write(filtered_text)

In [42]:
for i in range(37,151):
    text_file = f"text_files\{i}.txt"
    filtered_file = f"filtered_text_files\{i}.txt" 
    remove_stopwords(text_file,stopwords,filtered_file)

## Positive Score

In [17]:
with open(f"MasterDictionary\positive-words.txt",'r') as file:
    text = file.read()
positive_words = text.split()

In [21]:
def positive_score(text_file,positive_dict):
    with open(text_file,'r', encoding = 'utf-8') as file:
        text = file.read()
    words = text.split()
    positive_score = 0
    for word in words:
        word = word.lower()
        if word in positive_dict:
            positive_score += 1 
    return positive_score

In [22]:
filter_filenames = [f"filtered_text_files\{i}.txt" for i in range(37,151)]
positive_scores = []
for filename in filter_filenames:
    score = positive_score(filename, positive_words)
    positive_scores.append(score)
# print(positive_scores)

In [23]:
output_file["POSITIVE SCORE"] = positive_scores

## Negative Score

In [24]:
with open(r"MasterDictionary\negative-words.txt",'r', encoding = "utf-8") as file:
    text = file.read()
negative_words = text.split()

In [25]:
def negative_score(text_file,negative_dict):
    with open(text_file,'r',encoding= 'utf-8') as file:
        text = file.read()
    words = text.split()
    negative_score = 0
    for word in words:
        word = word.lower()
        if word in negative_dict:
            negative_score -= 1

    return negative_score * -1

In [26]:
negative_score = []
for filename in filter_filenames:
    score = positive_score(filename, negative_words)
    negative_score.append(score)

In [27]:
output_file["NEGATIVE SCORE"] = negative_score

## Polarity Score

In [28]:
def polarity_score(positive_score, negative_score):
    polarity_score = (positive_score - negative_score)/ ((positive_score + negative_score) + 0.000001)
    return polarity_score

In [29]:
print(polarity_score(output_file["POSITIVE SCORE"][0], output_file["NEGATIVE SCORE"][0]))

0.3827160446578266


In [30]:
positive_scores = list(output_file["POSITIVE SCORE"])
negative_scores = list(output_file["NEGATIVE SCORE"])

polarity_scores = []
for i in range(114):
    score = polarity_score(positive_scores[i], negative_scores[i])
    polarity_scores.append(score)

In [31]:
output_file["POLARITY SCORE"] = polarity_scores

## Subjectivity Score

In [34]:
def subjectivity_score(text_file,positive_score, negative_score):
    with open(text_file,'r', encoding = 'utf-8') as file:
        text = file.read()
    words = text.split()
    total_words = len(words)
    subjectivity_score = (positive_score + negative_score)/ ((total_words) + 0.000001)
    return subjectivity_score

In [35]:
subjectivity_scores = []
for i in range(len(filter_filenames)):
    score = subjectivity_score(filter_filenames[i], positive_scores[i], negative_scores[i])
    subjectivity_scores.append(score)

In [36]:
output_file["SUBJECTIVITY SCORE"] = subjectivity_scores

## Average Sentence Length 

In [40]:
def count_sentences(text):
    sentences = text.split(".")
    return len(sentences)

def count_words(text):
    words = text.split()
    return len(words)

def avg_sentence_length(text_file):
    with open(text_file,'r', encoding='utf-8') as file:
        text = file.read()
    no_of_words = count_words(text)
    no_of_sentences = count_sentences(text)
    avg_sentence_length = no_of_words / no_of_sentences
    return round(avg_sentence_length)

In [122]:
text_filenames = [f"text_files\{i}.txt" for i in range(37,151)]

In [42]:
avg_sen_lengths = []
for filename in text_filenames:
    length = avg_sentence_length(filename)
    avg_sen_lengths.append(length)

In [43]:
output_file["AVG SENTENCE LENGTH"] = avg_sen_lengths

## Percentage of complex words

In [78]:
def count_syllables(word):
    return len(
        re.findall('(?!e$)[aeiou]+', word, re.I) +
        re.findall('(?!es$)(?!ed$)[aeiou]', word, re.I) 
    )
    # (?!e$)[aeiou]+ pattern means to find vowel a,e,i,o,u in the word without the ending letter 'e'
    # (?!es$)(?!ed$)[aeiou] pattern is for handling the exception of 'es' and 'ed' ending words

In [79]:
def count_complex_words(filename):
    with open(filename,'r', encoding='utf-8') as file:
        text = file.read()
    words = text.split()
    complex_words = 0
    for word in words:
        if (count_syllables(word) > 2):
            complex_words += 1
    return complex_words

In [80]:
def percent_of_complex_words(filename):
    with open(filename,'r', encoding='utf-8') as file:
        text = file.read()
    no_of_words = count_words(text)
    no_of_complex_words = count_complex_words(filename)
    percent = round((no_of_complex_words / no_of_words)* 100, 3)
    
    return percent

In [81]:
p_of_complex_words = []
for filename in text_filenames:
    percent = percent_of_complex_words(filename)
    p_of_complex_words.append(percent)

In [82]:
output_file["PERCENTAGE OF COMPLEX WORDS"] = p_of_complex_words

## Fog Index

In [83]:
def fog_index(average_sentence_len, percent_of_complex_words):
    fog_index = round(0.4*(average_sentence_len + percent_of_complex_words), 3)
    return fog_index

In [84]:
fog_indexes = []
for i in range(114):
    index = fog_index(output_file["AVG SENTENCE LENGTH"][i], output_file["PERCENTAGE OF COMPLEX WORDS"][i])
    fog_indexes.append(index)

In [85]:
output_file["FOG INDEX"] = fog_indexes

In [86]:
output_file

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT
0,37,https://insights.blackcoffer.com/ai-in-healthc...,56,25,0.382716,0.078337,23,60.798,33.519,23,1067,1112
1,38,https://insights.blackcoffer.com/what-if-the-c...,45,28,0.232877,0.108148,20,49.536,27.814,20,694,740
2,39,https://insights.blackcoffer.com/what-jobs-wil...,57,32,0.280899,0.099330,20,57.049,30.820,20,955,977
3,40,https://insights.blackcoffer.com/will-machine-...,42,14,0.500000,0.075984,19,52.190,28.476,19,822,849
4,41,https://insights.blackcoffer.com/will-ai-repla...,48,18,0.454545,0.076834,21,51.571,29.028,21,870,916
...,...,...,...,...,...,...,...,...,...,...,...,...
109,146,https://insights.blackcoffer.com/blockchain-fo...,20,22,-0.047619,0.088983,16,54.835,28.334,16,482,513
110,147,https://insights.blackcoffer.com/the-future-of...,26,7,0.575758,0.057292,20,54.220,29.688,20,591,617
111,148,https://insights.blackcoffer.com/big-data-anal...,25,34,-0.152542,0.094551,15,51.733,26.693,15,582,653
112,149,https://insights.blackcoffer.com/business-anal...,23,2,0.840000,0.083056,21,55.735,30.694,21,311,319


## Average Number of Words Per Sentence

In [64]:
def avg_word_per_sentence(filename):
    with open(filename,'r', encoding='utf-8') as file:
        text = file.read()
    no_of_words = count_words(text)
    no_of_sentences = count_sentences(text)
    average = round(no_of_words / no_of_sentences)
    return average

In [65]:
avg_word_per_sentences = []
for filename in text_filenames:
    average = avg_word_per_sentence(filename)
    avg_word_per_sentences.append(average)

In [66]:
output_file["AVG NUMBER OF WORDS PER SENTENCE"] = avg_word_per_sentences

## Complex Word Count

In [67]:
no_of_complex_words = []
for filename in text_filenames:
    count = count_complex_words(filename)
    no_of_complex_words.append(count)

In [68]:
output_file["COMPLEX WORD COUNT"] = no_of_complex_words

## Word Count

### removing stopwords using nltk package

In [75]:
def get_word_count(text_file):
    """removing stopwords using nltk package"""
    with open(text_file,'r', encoding='utf-8') as file:
        text = file.read()
    tokens = word_tokenize(text)
    tokens = [w.lower() for w in tokens]
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in tokens]
    words = [word for word in stripped if word.isalpha()]
    stop_words = list(stopwords.words('english'))
    words = [w for w in words if not w in stop_words]
    wordcount = len(words)
    return wordcount

In [76]:
# word_count(r"text_files/37.txt")
word_count = []
for filename in text_filenames:
    count = get_word_count(filename)
    word_count.append(count)

In [77]:
output_file["WORD COUNT"] = word_count

## Syllable Per word

In [92]:
def syllable_per_word(filename):
    """ Returns average syllable per word"""
    with open(filename,'r', encoding = 'utf-8') as file:
        text = file.read()
    words = text.split()
    counts = []
    for word in words:
        counts.append(count_syllables(word))
    syllable_per_word = round(sum(counts)/ len(counts))

    return syllable_per_word

In [93]:
syllable_per_word_count = []
for filename in text_filenames:
    syllable_per_word_count.append(syllable_per_word(filename))

In [95]:
output_file["SYLLABLE PER WORD"] = syllable_per_word_count

## Personal Pronoun

In [112]:
def count_pronoun(filename):
    with open(filename, 'r', encoding='utf-8') as file:
        text = file.read()
    pattern = r"\b(I|we|my|ours|us)\b" 
    matches = len(re.findall(pattern, text))

    return matches

In [113]:
pronoun_count = []
for filename in text_filenames:
    pronoun_count.append(count_pronoun(filename))

In [115]:
output_file["PERSONAL PRONOUNS"] = pronoun_count

## Average Word Length

In [119]:
def count_characters(filename):
    with open(filename, 'r', encoding = 'utf-8') as file:
        text = file.read()
    text = text.replace(" ", "")
    
    return len(text)

In [124]:
def avg_word_length(filename):
    with open(filename, 'r',encoding= 'utf-8') as file:
        text = file.read()
    num_of_char = count_characters(filename)
    num_of_words = count_words(text)

    return int(num_of_char/num_of_words)

In [125]:
word_length_count = []
for filename in text_filenames:
    word_length_count.append(avg_word_length(filename))

In [127]:
output_file["AVG WORD LENGTH"] = word_length_count

# Saving into CSV file 

In [130]:
output_file.to_csv("Output.csv", index= False)